In [71]:
import numpy as np
import chemical_potential
import vapour_pressure as vp
import BentheimerSandstone

### 水分状態を換算するクラス  
換算する前の水分状態と換算後の水分状態を指定し、温度と水分状態を与えることで値を換算するクラス  
$T$：絶対温度[K]  
$R_v$：水蒸気の気体定数  
$RH$：相対湿度[-]  
$P_v$：水蒸気圧[Pa]  
$P_{vs}$：飽和水蒸気圧[Pa]  
$\mu$：水分化学ポテンシャル[J/kg]  

RH（相対湿度） - Pv（水蒸気圧）　関係  
$RH = \frac{P_v}{P_{vs}}$  

RH（相対湿度） - Miu（水分化学ポテンシャル）　関係  
$\mu = R_v T log(RH)$

In [72]:
### 水分状態を換算するクラス ######################
class MoistureConversion():
    Rv = 8316.0/18.016
    def __init__(self, moistureIndexBefore, moistureIndexAfter):
        self.B = moistureIndexBefore
        self.A = moistureIndexAfter
        
    def convert(self, tem, moisture):
#   RH（相対湿度） - Pv（水蒸気圧）　関係
        if self.B == 'RH' and self.A =='Pv':
            pv = moisture* vp.Pvs(tem) 
            return pv
        elif self.B == 'Pv' and self.A =='RH':
            rh = moisture/ vp.Pvs(tem) 
            return rh
#   RH（相対湿度） - Miu（水分化学ポテンシャル）　関係
        elif self.B == 'RH' and self.A =='Miu':
            miu = self.Rv * tem* np.log(moisture) 
            return miu
        elif self.B == 'Miu' and self.A =='RH':
            rh = np.exp(moisture/(self.Rv*tem))
            return rh
#   Pv（水蒸気圧） - Miu（水分化学ポテンシャル）　関係
        elif self.B == 'Pv' and self.A =='Miu':
            miu = self.Rv * tem* np.log(moisture/vp.Pvs(tem)) 
            return miu
        elif self.B == 'Miu' and self.A =='Pv':
            pv = vp.Pvs(tem)*np.exp(moisture/(self.Rv*tem))
            return pv
#   換算が要らない場合
        elif self.B == self.A:
            return moisture
        else : 
            raise ValueError('各物性クラスのmoistureIndexには、「RH」か「Pv」,「Miu」を指定しください。')
################################################

In [73]:
# 使用例
c = MoistureConversion('Miu','RH')
c.convert(293.15, -1500.0)

0.988975971012088

### 物性値を取得するクラス  
各物性値ファイルに存在する物性値を元の形式のまま取得してくるクラス。  

#### 実験から熱水分物性値として考えられるもの  
【参考】建築材料の熱・空気・湿気物性値，日本建築学会，丸善株式会社，2001など

#### 熱物性値  
$q$：単位時間当たりの熱流量の密度[W/m2]  
$\rho_0$：密度[kg/m3]  
$c_0$：比熱[J/kg K]  
$c_0\rho_0$：容積熱容量[J/m3 K]  

$\lambda$：熱伝導率[W/mK]  
定義：$q = -\lambda・grad\space T$  

$R$：熱抵抗[Km2/W]  

#### 水分物性値  
$m_v$：単位時間当たりの水蒸気流量の密度[kg/m2 s]  
$m_m$：単位時間当たりの水分流量の密度[kg/m2 s]  
$\phi$：含水率[m3/m3]  
他標記として$w$[kg/m3]、$u$[kg/kg]など  
$S$：飽和度[-]  
#### 気相水分伝導率  
$\delta_p$：透湿率[kg/m Pa s]　文中記号DP  
定義：$m_v = -\delta_p・grad\space p_v$  

$Z_p$：湿気抵抗[m2Pa s/kg]  
$\delta_v$：水蒸気の拡散係数[m2/s]  
定義：$m_v = -\delta_v・grad\space \rho_v$

$k_T$：温度勾配水分伝導率[kg/m K s]  
定義：$m_m = -k_T・grad\space T$  

$D_T$：温度勾配水分拡散係数[m2/K s]  
定義：$m_m = -\rho_0 D_T・grad\space T$  

$\lambda^{'}_{\mu g}$：水分化学ポテンシャル勾配に対する気相水分伝導率[kg/ms (J/kg)]  
定義：$m_v = -\lambda^{'}_{\mu g}・grad\space \mu$

$\lambda^{'}_{Tg}$：温度勾配に対する気相水分伝導率[kg/ms K]  
定義：$m_v = -\lambda^{'}_{Tg}・grad\space \mu$

#### 液相水分伝導率  
$k_m$：水分伝導率[kg/m Pa s]  
定義：$m_m = -k_m・grad\space s$

$D_w$：水分拡散係数[m2/s]  
定義：$m_m = -\rho_0 D_w・grad\space u$  

$\lambda^{'}_{\mu l}$：水分化学ポテンシャル勾配に対する液相水分伝導率[kg/ms (J/kg)]  
定義：$m_v = -\lambda^{'}_{\mu l}・grad\space \mu$

#### 空気の拡散係数  
$k_a$：透気率[m2/Ks]  


※新しい基準で物性値を測定した場合、本クラスに書き加えること


In [131]:
### ある水分状態から物性値を読み取る（換算は無し）
class GetMaterial():

    # 新しく物性モジュールを作成した場合は、ここに追加してください。
    mlist = {
        '合板' : 'Plywood', #物性値ファイルの名前
        'グラスウール' : 'GlassWool',
        'BentheimerSandstone': 'BentheimerSandstone'
    }
    
    # 問題は取ってきた物性値が何であるか（何に対する液相・気相水分伝導率であるか）
    
    def __init__(self, material,mainMoistureIndex):
        if self.mlist[material] == 'BentheimerSandstone':
            self.material = BentheimerSandstone.BentheimerSandstone()
        elif self.mlist[material] == 'Plywood':
            self.material = 0.0       #ダミー
        else : 
            raise ValueError('各物性クラスのmaterialには、mlistおよびif文内に加えたstrを指定しください。')

        self.list = self.material.component
        self.B = mainMoistureIndex
#        self.A = material.moistureIndex
#        self.MC = MoistureConversion(self.B, self.A)


In [132]:
# 使用例
stone = GetMaterial('BentheimerSandstone',0)

print(stone.list)
#print(BentheimerSandstone.BentheimerSandstone().component)
#print(stone.material,stone.list)

['crow', 'LAM', 'Phi', 'Miu', 'DL', 'LDP', 'DPhi']


### 取得した物性値を流量計算に適した形に換算するクラス

In [129]:
class PropertyConversion(GetMaterial):
    
    row = 1000.0
    grav = 9.806650
    
##########################################################
#   熱容量
    def crow(self, tem, moisture):
        if 'crow' in self.list:
            return self.material.crow( tem, self.MC.convert(self, tem, moisture) )
#   熱伝導率
    def LAM(self, tem, moisture ):
        if 'LAM' in self.list:
            return self.material.LAM( tem, self.MC.convert(self, tem, moisture) )
    
##########################################################
#   水分容量（気相の水分状態（ポテンシャル・相対湿度など）から含水率への変換）
    def phi(self, tem, moisture):
        return self.material.phi( tem, self.MC.convert(self, tem, moisture) )
#   含水率から気相の水分状態（ポテンシャル・相対湿度など）への変換）
    def moisture(self, tem, phi):
        return self.material.moisture( tem, phi )
    
#   水分化学ポテンシャル勾配に対する液相水分伝導率
    def LDML(self, tem, moisture):
        if 'LDML' in self.list:
            return self.material.LDML( tem, self.MC.convert(self, tem, moisture) )
        elif 'Dw' in self.list:
            return row / grav * self.material.Dw( tem, self.MC.convert(self, tem, moisture) ) #ダミー　値を換算する
        
#   水分化学ポテンシャル勾配に対する気相水分伝導率
    def LDMG(self, tem, moisture):
        if 'LDMG' in self.list:
            return self.material.LDMG( tem, phi )
        elif 'DP' in self.list:
            RH   = chemical_potential.RH_cal(Tem,Miu)  #ダミー 
            DPDM = chemical_potential.DPDM_cal(Tem,RH)#ダミー
            LDMG = LDP*DPDM#ダミー
            return LDMG#ダミー
        
#   温度勾配に対する気相水分伝導率
    def LDTG(self, tem, moisture):
        if 'LDTG' in self.list:
            return self.material.LDMG( tem, phi )
        elif 'DP' in self.list:    
            RH   = chemical_potential.RH_cal(Tem,Miu) #仮
            DPDT = chemical_potential.DPDT_cal(Tem,RH)#仮
            LDTG = LDP*DPDT#仮
            return LDTG#仮
    
#   含水率の水分化学ポテンシャル微分
    def DPhi(self, tem,moisture):
        return self.material.DPhi(tem, self.MC.convert(self, tem, moisture))

################################################      
